# Download L3 matchup data from NASA's sensors using ERDDAP's Interpolate Tool

**Last updated: 30/04/2024**

[**NOAA's CoastWatch Data Portal**](https://coastwatch.noaa.gov/cwn/data-access-tools/coastwatch-data-portal.html) provides acces to **ERDDAP data server**, a tool that allows the creation of **L3** satellite **matchup** data for the most popular sensors. This script is configured to download L3 matchup data from **Aqua-MODIS** and **SNPP-VIIRS** (daily, 4 km resolution) for a dataset of in situ **HPLC** for the North Sea provided by CEFAS. The MERIS and NOAA20-VIIRS sensors are not used as their operational time does not match that of HPLC data availability. [**ERDDAP's Interpolate Tool**](https://umd.instructure.com/courses/1336575/pages/erddap-tutorial?module_item_id=11631927) will allow us to perform the matchups.

[ERDDAP](https://k-rns.github.io/workshop_data_reuse/aio/index.html) is a tool that provides a consistent way to subset and download gridded (Level 3 and above) environmental datasets in common file formats with options to make graphs and maps. Various institutions have installed ERDDAP to allow their users to visualise and download data (e.g., NOAA CoastWatch Central Operations, NASA Ocean Biology (OB.DAAC), EUMETSAT, etc.) which means that there are many erddap servers around. Users can search two types of data, also called protocols: tabledap & griddap. Users can subset a dataset based on constraining the variables. 

NOAA's ERDDAP server is [here](https://coastwatch.pfeg.noaa.gov/erddap/index.html) and you can find an overview of NOAA's ERDDAP capabilities [here](https://umd.instructure.com/courses/1336575/files/71462008?module_item_id=11824952). NOAA's ERDDAP server is part of the CoastWatch program, which offers other [data tools](https://coastwatch.noaa.gov/cwn/data-access-tools.html) interesting for visualisation. We are going to use ERDDAP's [Advanced Search](https://coastwatch.pfeg.noaa.gov/erddap/search/advanced.html?page=1&itemsPerPage=1000) to explore all the chlorophyll data products available at NOAA. We will take advantage of ERDDAP’s RESTful web services and create and ERDDAP URL for searching for data products. Once we select the products we are interested in, we will do the matchups.

This script was developed using this [Python Notebook](https://github.com/rsignell-usgs/notebook/blob/master/ERDDAP/ERDDAP_advanced_search_test.ipynb/), which is recommended in the ERDDAP griddap [documentation](https://coastwatch.pfeg.noaa.gov/erddap/griddap/documentation.html) provided by NOAA's CoastWatch. Notice that it uses URL percent encoding. You can find the meanings of those encodings [here](https://www.eso.org/~ndelmott/url_encode.html).

**You can use this script as a template and modify the code sections below to customise it for your own area of study and datasets.**

## Import libraries and define paths

In [11]:
import os
from pathlib import Path
import pandas as pd
from pandas import DataFrame, read_csv
from datetime import datetime, timedelta

In [12]:
# Create a download directory for our outputs
PATH_ROOT_DIR = Path.cwd().resolve().parents[1] # /absolute/path/to/two/levels/up
NAME_DOWNLOAD_DIR_HPLC_MATCHUPS = 'data_matchups_HPLC_NASA_csv'
full_path_download_dir_hplc = os.path.join(PATH_ROOT_DIR,"data","raw","NASA_data",NAME_DOWNLOAD_DIR_HPLC_MATCHUPS)
os.makedirs(full_path_download_dir_hplc, exist_ok=True)

## Read in our in situ HPLC observations

In [13]:
# Read the HPLC data file from CEFAS (AR: file created by the Matlab function prepareHPLCdata.m)
NAME_HPLC_DATA_FILE = 'cefasHPLCfiltered.csv'
full_path_hplc_data_dir = os.path.join(PATH_ROOT_DIR, 'data', 'processed', NAME_HPLC_DATA_FILE)
matchup_hplc_locations_list = pd.read_csv(full_path_hplc_data_dir, sep = ',')

# Converting date column into right format
matchup_hplc_locations_list["DateTime"] = pd.to_datetime(matchup_hplc_locations_list["DateTime"])
matchup_hplc_locations_list # print to the screen

,idd,Survey_name,Station_number,Prime_number,DateTime,Latitude,Longitude,Smartbuoy,Sample_depth,TP_ug_L,...,Lut_ug_L,Myxo_ug_L,Croc_ug_L,x19_Keto_Hex_fuco_ug_L,Hexkfuco_ug_L,HexkfucoL_ug_L,x4keto_hex_ug_L,x4keto_hexL_ug_L,bathymetry_m,season
0,625,CEND19_17,230.0,102.0,2017-10-28 01:18:00,48.350783,-5.750117,NaN,4,0.701378,...,0.000000,0.000000,0.00000,NaN,NaN,NaN,0.000000,NaN,-116.982548,Autumn
1,671,CEND17_18,169.0,102.0,2018-10-25 03:42:00,48.366280,-5.725660,NaN,6,0.796561,...,0.000000,NaN,0.00000,NaN,NaN,NaN,0.004086,NaN,-116.179468,Autumn
2,672,CEND17_18,176.0,106.0,2018-10-25 20:40:00,48.547920,-4.928820,NaN,6,0.628834,...,0.000662,NaN,0.00000,NaN,NaN,NaN,0.002824,NaN,-74.689037,Autumn
3,626,CEND19_17,231.0,106.0,2017-10-28 05:48:00,48.552300,-4.915950,NaN,4,0.858928,...,0.000000,0.000000,0.00000,NaN,NaN,NaN,0.000000,NaN,-77.795798,Autumn
4,55,CEND09_11,44.0,NaN,2011-05-22 15:50:00,48.778933,-4.390117,NaN,5,1.906309,...,0.004340,0.003500,0.00701,NaN,NaN,NaN,NaN,NaN,-88.030648,Spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,499,CEND15_13,191.0,72.0,2013-08-29 12:40:00,61.232000,-0.401000,NaN,4,0.620383,...,0.001790,0.000000,NaN,0.00000,0.0,0.0,NaN,NaN,-164.470400,Summer
670,166,CEND13_10,160.0,74.0,2010-09-01 04:06:00,61.251333,1.393667,NaN,3,6.715475,...,0.024246,0.013840,NaN,0.11529,NaN,NaN,NaN,NaN,-148.207900,Summer
671,445,CEND18_15,125.0,73.0,2015-08-30 14:47:00,61.285900,0.488133,NaN,4,2.892239,...,0.014518,0.000000,0.00000,NaN,NaN,NaN,0.016483,NaN,-167.815296,Summer
672,533,CEND18_16,120.0,73.0,2016-08-29 09:35:00,61.288283,0.500183,NaN,4,2.378788,...,0.001587,0.006305,0.00000,NaN,NaN,NaN,0.000000,NaN,-168.083801,Summer


In [14]:
# Extract the data that we need from the CEFAS .csv file
LIST_OBS_CHLA = matchup_hplc_locations_list.TChlA_ug_L[:]
LIST_OBS_LON = matchup_hplc_locations_list.Longitude[:]
LIST_OBS_LAT = matchup_hplc_locations_list.Latitude[:]
LIST_OBS_DATETIME = matchup_hplc_locations_list.DateTime[:]

# Format time
LIST_OBS_DATETIME = pd.to_datetime(LIST_OBS_DATETIME)
LIST_OBS_DATETIME = LIST_OBS_DATETIME.dt.strftime("%Y-%m-%dT%H:%M:%SZ")

In [15]:
# Spatio-temporal box where in situ data entries sit (based on scanning the dataset)
LAT_MIN = 48.0
LAT_MAX = 63.0
LON_MIN = -11.0
LON_MAX = 9.0
START_DATETIME = "2000-01-01T00:00:00Z"
END_DATETIME   = "2010-11-01T00:00:00Z"

## Explore the available datasets in NOAA's product list using ERDDAP's Advanced Search

In [16]:
# Build the ERDAPP URL, which will return output in .csv form

METADATA_KEYWORDS = "chlorophyll+global+4km+l3" # +%22Science+Quality%22 (suppress so that ESA products are also listed)
PROTOCOL = 'griddap' # protocols: griddap or tableddap
CATEGORY = "ocean_color" # available categories: https://coastwatch.noaa.gov/erddap/categorize/ioos_category/index.html?page=1&itemsPerPage=1000

base = (
    'https://coastwatch.pfeg.noaa.gov/erddap/search/index.csv' # index
    '?page=1'
    '&itemsPerPage=1000'
    '&searchFor={}'
    '&protocol={}'
    '&cdm_data_type=(ANY)'
    '&institution=(ANY)'
    '&ioos_category={}'
    '&keywords=(ANY)'
    '&long_name=(ANY)'
    '&standard_name=(ANY)'
    '&variableName=(ANY)'
    '&maxLat={}'
    '&minLon={}'
    '&maxLon={}'
    '&minLat={}'
    '&minTime={}'
    '&maxTime={}').format

url = base(
    METADATA_KEYWORDS,
    PROTOCOL,
    CATEGORY,
    LAT_MAX,
    LON_MIN,
    LON_MAX,
    LAT_MIN,
    START_DATETIME,
    END_DATETIME
)

dft = pd.read_csv(url, usecols=['Title', 'Summary', 'Institution', 'Dataset ID'])

In [17]:
print('Chlorophyll datasets found = {}'.format(len(dft)))
print(url)
dft

Chlorophyll datasets found = 130
https://coastwatch.pfeg.noaa.gov/erddap/search/index.csv?page=1&itemsPerPage=1000&searchFor=chlorophyll+global+4km+l3&protocol=griddap&cdm_data_type=(ANY)&institution=(ANY)&ioos_category=ocean_color&keywords=(ANY)&long_name=(ANY)&standard_name=(ANY)&variableName=(ANY)&maxLat=63.0&minLon=-11.0&maxLon=9.0&minLat=48.0&minTime=2000-01-01T00:00:00Z&maxTime=2010-11-01T00:00:00Z


,Title,Summary,Institution,Dataset ID
0,"Chlorophyll-a, Aqua MODIS, NPP, L3SMI, Global,...",Moderate Resolution Imaging Spectroradiometer ...,NASA/GSFC OBPG,erdMH1chla1day_R2022NRT
1,"Chlorophyll-a, Aqua MODIS, NPP, L3SMI, Global,...",Moderate Resolution Imaging Spectroradiometer ...,NASA/GSFC OBPG,erdMH1chla8day_R2022NRT
2,"Chlorophyll-a, Aqua MODIS, NPP, L3SMI, Global,...",Moderate Resolution Imaging Spectroradiometer ...,NASA/GSFC OBPG,erdMH1chlamday_R2022NRT
3,"Chlorophyll-a, Aqua MODIS, NPP, L3SMI, Global,...",Moderate Resolution Imaging Spectroradiometer ...,NASA/GSFC OBPG,erdMH1chla1day_R2022SQ
4,"Chlorophyll-a, Aqua MODIS, NPP, L3SMI, Global,...",Moderate Resolution Imaging Spectroradiometer ...,NASA/GSFC OBPG,erdMH1chla8day_R202SQ
...,...,...,...,...
125,ESA CCI Ocean Colour Product (CCI ALL-v6.0-MON...,Data products generated by the Ocean Colour co...,Plymouth Marine Laboratory,pmlEsaCCI60OceanColorMonthly_Lon0360
126,ESA CCI Ocean Colour Product (CCI ALL-v5.0-DAI...,Data products generated by the Ocean Colour co...,Plymouth Marine Laboratory,pmlEsaCCI50OceanColorDaily
127,ESA CCI Ocean Colour Product (CCI ALL-v5.0-DAI...,Data products generated by the Ocean Colour co...,Plymouth Marine Laboratory,pmlEsaCCI50OceanColorDaily_Lon0360
128,ESA CCI Ocean Colour Product (CCI ALL-v6.0-DAI...,Data products generated by the Ocean Colour co...,Plymouth Marine Laboratory,pmlEsaCCI60OceanColorDaily


We want Standard (science quality) products, as opposed to Near-Real-Time. The datasets that we will download are:
- **nesdisVHNSQchlaDaily**: VIIRS, global, 4 km
- **erdMH1chla1day**: Aqua-MODIS, global, 4 km

## Download the products of interest using ERDDAP's Interpolate Tool

ERDDAP has the ability to interpolate data (https://erddap.ioos.us/erddap/convert/interpolate.html). The Interpolate Tool limits the URL length to 100 rows of input. Even with this limit, this tool is much more efficient than making a separate request for each source Time Latitude Longitude value because bulk requests have 1/100th the network overhead and also get the data from the source dataset much more efficiently (often almost 100 times more efficiently).

In [18]:
# Download parameters

BATCH_SIZE = 100
num_batches = len(LIST_OBS_LAT) // BATCH_SIZE + (len(LIST_OBS_LAT) % BATCH_SIZE > 0)

LIST_DATASET_IDS = [
    "nesdisVHNSQchlaDaily",
    "erdMH1chla1day",
]

# Variable names can be found under the "Summary" section of the products (see printed table above)
LIST_VARIABLE_NAMES = [
    "chlor_a",
    "chlorophyll",
]

LIST_OUTPUT_NAMES = [
    "viirssnpp_4km",
    "aquamodis_4km"
]

# Algorithm options are listed in the documentation of the Interpolate Tool: https://erddap.ioos.us/erddap/convert/interpolate.html
ALGORITHM = "Nearest" #"Bilinear"
NEARBY = 4

The use of the "Bilinear" method yields 165 matchups whereas the "Nearest" method yields 188 matchups. We are going to stick to the "Nearest" method to be consistent with the method used in other matchups analyses (e.g., OC-CCI L3 matchups).

In [19]:
%%time

# Loop through datasets
for dataset_id, variable_name, output_name in zip(LIST_DATASET_IDS, LIST_VARIABLE_NAMES, LIST_OUTPUT_NAMES):
    
    print("This is dataset ID: ", dataset_id)
    
    # Initialise a list
    list_of_matchups = []
    
    # Loop through the batches
    for batch_index in range(num_batches):
    
        start_index = batch_index * BATCH_SIZE
        end_index = (batch_index + 1) * BATCH_SIZE
        print("Start index: ", start_index)
        print("End index: ",   end_index)

        # Extract a group of 100 items from each list
        lat_batch = LIST_OBS_LAT[start_index:end_index]
        lon_batch = LIST_OBS_LON[start_index:end_index]
        datetime_batch = LIST_OBS_DATETIME[start_index:end_index]
    
        insitu_batch_data = {
            'time': datetime_batch,
            'latitude': lat_batch,
            'longitude': lon_batch
        }

        df_batch = pd.DataFrame(insitu_batch_data)
    
        # Convert time to the desired format
        df_batch['time'] = pd.to_datetime(df_batch['time']).dt.strftime('%Y-%m-%dT%H%%3A%M%%3A%SZ')

        # Create the final formatted string with a regular comma separator
        csv_batch = df_batch.to_csv(index=False, header="time,latitude,longitude", lineterminator='%0A', sep=',') 

        # Replace commas with %2C and delete double quotes
        formatted_time_lat_lon_table = csv_batch.replace(',', '%2C')
        formatted_time_lat_lon_table = formatted_time_lat_lon_table.replace('"', '')

        # Build the downlaod URL
        base = (
            'https://coastwatch.pfeg.noaa.gov/erddap/convert/interpolate.csv'
            '?TimeLatLonTable={}'
            '&requestCSV={}' # datasetID
            '%2F{}' # variable name
            '%2F{}' # algorithm
            '%2F{}' # nearby
        ).format 

        url = base(
            formatted_time_lat_lon_table,
            dataset_id,
            variable_name,
            ALGORITHM,
            NEARBY
        )
        print(url)
    
        # Import your dataset into pandas
        df_downloaded_data = pd.read_csv(url)
    
        # Append
        list_of_matchups.append(df_downloaded_data)
         
    # Concatenate all downloaded DataFrames into a single DataFrame and save the final DataFrame into a csv file
    df_matchups = pd.concat(list_of_matchups, ignore_index=True)
    csvfilename = os.path.join(full_path_download_dir_hplc, f"{output_name}_L3_matchups.csv")
    df_matchups.to_csv(csvfilename, mode='w', header=['time','latitude','longitude','chl'], index=False)
    
    print('Number of matchups found: ', df_matchups.iloc[:,3].count()) # count number non-NaN elements
    print(df_matchups.head())

print("Download completed!")

This is dataset ID:  nesdisVHNSQchlaDaily
Start index:  0
End index:  100
https://coastwatch.pfeg.noaa.gov/erddap/convert/interpolate.csv?TimeLatLonTable=time%2Clatitude%2Clongitude%0A2017-10-28T01%3A18%3A00Z%2C48.35078333%2C-5.750116667%0A2018-10-25T03%3A42%3A00Z%2C48.36628%2C-5.72566%0A2018-10-25T20%3A40%3A00Z%2C48.54792%2C-4.92882%0A2017-10-28T05%3A48%3A00Z%2C48.5523%2C-4.91595%0A2011-05-22T15%3A50%3A00Z%2C48.77893333%2C-4.390116667%0A2018-04-08T11%3A15%3A00Z%2C48.79476667%2C-4.5661%0A2018-04-09T11%3A11%3A00Z%2C48.97823333%2C-5.774533333%0A2018-10-17T20%3A16%3A00Z%2C49.00931667%2C-6.607083333%0A2018-10-24T22%3A28%3A00Z%2C49.02274%2C-5.85338%0A2017-10-27T19%3A33%3A00Z%2C49.03015%2C-5.836866667%0A2018-10-27T02%3A10%3A00Z%2C49.19888%2C-3.95008%0A2018-10-26T00%3A57%3A00Z%2C49.24044%2C-5.04406%0A2011-05-22T08%3A37%3A00Z%2C49.25181667%2C-3.686016667%0A2018-04-06T10%3A59%3A00Z%2C49.25291667%2C-2.348533333%0A2011-05-25T04%3A07%3A00Z%2C49.28416667%2C-5.10065%0A2011-05-25T04%3A09%3A00Z%2C49.2

In [20]:
# Initialise a new dataframe from the input one
selected_columns = ['DateTime', 'Latitude', 'Longitude', 'Sample_depth']
df_coords_with_data = matchup_hplc_locations_list[selected_columns].copy()

# Pick up and add the variable from every saved dataframe
for output_name in LIST_OUTPUT_NAMES:
    csvfilename = os.path.join(full_path_download_dir_hplc, f"{output_name}_L3_matchups.csv")
    if os.path.exists(csvfilename):
        df_file = pd.read_csv(csvfilename)
        variable_column = df_file["chl"]
        df_coords_with_data[f"{output_name}"] = variable_column

# Save new dataframe with all data
csvfilename = os.path.join(full_path_download_dir_hplc, "nasa_hplc_matchups.csv")
df_coords_with_data.to_csv(csvfilename)
df_coords_with_data

,DateTime,Latitude,Longitude,Sample_depth,viirssnpp_4km,aquamodis_4km
0,2017-10-28 01:18:00,48.350783,-5.750117,4,NaN,NaN
1,2018-10-25 03:42:00,48.366280,-5.725660,6,NaN,NaN
2,2018-10-25 20:40:00,48.547920,-4.928820,6,NaN,NaN
3,2017-10-28 05:48:00,48.552300,-4.915950,4,NaN,NaN
4,2011-05-22 15:50:00,48.778933,-4.390117,5,NaN,NaN
...,...,...,...,...,...,...
669,2013-08-29 12:40:00,61.232000,-0.401000,4,0.232571,0.28404
670,2010-09-01 04:06:00,61.251333,1.393667,3,NaN,NaN
671,2015-08-30 14:47:00,61.285900,0.488133,4,NaN,NaN
672,2016-08-29 09:35:00,61.288283,0.500183,4,NaN,NaN
